In [1]:
import cv2
from scipy import optimize as opt
import numpy as np
import sys
import math

In [2]:
np.random.seed(2020)
is_planner = False

In [3]:
image_files = ['chess-1.jpg'] if is_planner==False else ['chess-1.jpg', 'chess-2.jpg', 'chess-3.jpg']
correspondence_points_files = ['correspondence_points-1.txt'] if is_planner==False else ['correspondence_points-1.txt', 'correspondence_points-2.txt', 'correspondence_points-3.txt']
ransac_config_file = 'RANSAC.config'

In [4]:
def load_image(image_file):
    image = cv2.imread(image_file)
    return image;

# Part-1

In [5]:
def feature_extraction(image_files, correspondence_points_files):
    
    ctr = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
    for image_file, correspondence_points_file in zip(image_files, correspondence_points_files):
                
        image = load_image(image_file=image_file)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        world_points = np.zeros((7*7,3), np.float32)
        world_points[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
        
        ret, img_points = cv2.findChessboardCorners(gray_image, (7,7), cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

        if ret:
            img_points_2 = cv2.cornerSubPix(gray_image, img_points, (11,11), (-1,-1), ctr)
            image = cv2.drawChessboardCorners(image, (7,7), img_points_2, ret)
            
            file = open(correspondence_points_file, 'w')
            for wp, ip in zip(world_points, img_points.reshape(-1,2)):
                wp = wp * 70
                file.write(f'{str(wp[0])} {str(wp[1])} {str(wp[2])} {str(ip[0])} {str(ip[1])}\n')
            file.close()
        cv2.imshow(f'{image_file}', image)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
feature_extraction(image_files=image_files, correspondence_points_files=correspondence_points_files)
for correspondence_points_file in correspondence_points_files:
    with open(correspondence_points_file, 'r') as file:
        print(f'correspondence points file : {correspondence_points_file}')
        for line in file:
            print(line)
    file.close()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wef8acrf\opencv\modules\highgui\src\window.cpp:651: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


# Part-2

In [ ]:
def get_correspondence_data(correspondence_points_files=correspondence_points_files):
    
    world_points = list()
    img_points = list()
    
    for correspondence_points_file in correspondence_points_files:
        wps = list()
        ips = list()
        with open(correspondence_points_file, 'r') as file:
            for line in file:
                points = line.split()
                wps.append([float(points[point]) for point in range(0,3,1)])
                ips.append([float(points[point]) for point in range(3,5,1)])
        
        world_points.append(wps)
        img_points.append(ips)
    
    world_points = np.array(world_points).astype('float32')
    img_points = np.array(img_points).astype('float32')
    
    return world_points, img_points

In [ ]:
def get_matirx_A_NP(world_points, img_points):
    A = list()
    for wp,ip in zip(world_points, img_points):
        pi = list()
        xi, yi = ip[0], ip[1]
        eq1, eq2 = list(), list()

        pi.extend(wp)
        pi.append(1)
        
        eq1.extend(pi)
        eq1.extend([0,0,0,0])
        eq1.extend([-1*xi*p for p in pi])
        
        eq2.extend([0,0,0,0])
        eq2.extend(pi)
        eq2.extend([-1*yi*p for p in pi])
        
        eq1, eq2 = np.array(eq1), np.array(eq2)
        
        A.append(eq1)
        A.append(eq2)

    return np.array(A)

def get_matrix_M_NP(A):
    
    U, D, VT = np.linalg.svd(A, full_matrices = True)
    M = VT[-4].reshape(3, 4)
    
    a1 = M[0][:3].T
    a2 = M[1][:3].T
    a3 = M[2][:3].T
    b = M[:,3].reshape((3,1))
    return a1, a2, a3, b, M

def compute_parameters_NP(a1, a2, a3, b):
    rho_abs = 1 / np.linalg.norm(a3.T)
    u0 = (rho_abs ** 2) * (a1.T.dot(a3))
    v0 = (rho_abs ** 2) * (a2.T.dot(a3))
    alpha_v = np.sqrt(((rho_abs ** 2) * a2.T.dot(a2)) - (v0 ** 2))
    s = ((rho_abs ** 4) / alpha_v) * np.cross(a2.T, a3.T).dot(np.cross(a1.T, a3.T).T)
    alpha_u = np.sqrt(((rho_abs ** 2) * a1.T.dot(a1)) - (s ** 2) - (u0 ** 2))
    K_star = np.array([[alpha_u, s, u0],[0, alpha_v, v0],[0, 0, 1]])
    E = np.sign(b[2])
    T_star = E * rho_abs * np.linalg.inv(K_star).dot(b)
    r3 = E * rho_abs * a3
    r1 = ((rho_abs ** 2) / alpha_v) * np.cross(a1, a3)
    r2 = np.cross(r3, r1)
    R_star = np.array([r1.T, r2.T, r3.T]).T
    M = rho_abs * np.array([[a1[0], a1[1], a1[2], b[0][0]],[a2[0], a2[1], a2[2], b[1][0]],[a3[0], a3[1], a3[2], b[2][0]]])
    return u0, v0, alpha_u, alpha_v, s, K_star, T_star, R_star, M

In [ ]:
def get_matirx_A_P(world_points, img_points):
    A = list()
    
    for wps,ips in zip(world_points, img_points):
        a = list()
        
        for wp,ip in zip(wps, ips):
            pi = list()
            xi, yi = ip[0], ip[1]
            eq1, eq2 = list(), list()
            
            pi.extend(wp[:2])
            pi.append(1)
            
            eq1.extend(pi)
            eq1.extend([0, 0, 0])
            eq1.extend([-1*xi*p for p in pi])
            
            eq2.extend([0, 0, 0])
            eq2.extend(pi)
            eq2.extend([-1*yi*p for p in pi])
            
            eq1, eq2 = np.array(eq1), np.array(eq2)
            
            a.append(eq1)
            a.append(eq2)
        
        A.append(np.array(a))
    
    return np.array(A)

def get_matrix_H_P(A):
    
    H = list()
    
    for a in A:
        U, D, VT = np.linalg.svd(a)
        h = VT[np.argmin(D)].reshape(3, 3)        
        H.append(h)
    
    return np.array(H)

def get_matrix_V_P(H):
    
    V = list()
    
    for h in H:
        V00T = np.array([ h[0, 0]*h[0, 0], h[0, 0]*h[1, 0] + h[1, 0]*h[0, 0], h[1, 0]*h[1, 0], h[2, 0]*h[0, 0] + h[0, 0]*h[2, 0], h[2, 0]*h[1, 0] + h[1, 0]*h[2, 0], h[2, 0]*h[2, 0] ])
        V11T = np.array([ h[0, 1]*h[0, 1], h[0, 1]*h[1, 1] + h[1, 1]*h[0, 1], h[1, 1]*h[1, 1], h[2, 1]*h[0, 1] + h[0, 1]*h[2, 1], h[2, 1]*h[1, 1] + h[1, 1]*h[2, 1], h[2, 1]*h[2, 1] ])
        V01T = np.array([ h[0, 0]*h[0, 1], h[0, 0]*h[1, 1] + h[1, 0]*h[0, 1], h[1, 0]*h[1, 1], h[2, 0]*h[0, 1] + h[0, 0]*h[2, 1], h[2, 0]*h[1, 1] + h[1, 0]*h[2, 1], h[2, 0]*h[2, 1] ])
        
        V1 = V01T
        V2 = np.subtract(V00T , V11T)

        V.append(V1)
        V.append(V2)
    
    return np.array(V)

def compute_parameters_P(H, V):
    
    U, D, VT = np.linalg.svd(V, full_matrices = True)
    S = VT[np.argmin(D)]
        
    c1 = (S[1]*S[3]) - (S[0]*S[4])
    c2 = (S[0]*S[2]) - (S[1]**2)
    v0 = c1 / c2
    lamda = S[5] - (((S[3]**2) + (v0*c1))/S[0])
    alpha_u = np.sqrt(lamda/S[0])
    alpha_v = np.sqrt((lamda*S[0])/c2)
    s = (-1 * S[1] * (alpha_u ** 2) * alpha_v) / lamda
    u0 = ((s*v0)/alpha_u) - ((S[3]*(alpha_u**2))/lamda)
    K_star = np.array([[alpha_u, s, u0],[0, alpha_v, v0],[0, 0, 1]])
    
    T_star = list()
    R_star = list()
    M = list()
    
    for h in H:
        h1 = h[:,0]
        h2 = h[:,1]
        h3 = h[:,2]
        
        alpha_abs = 1 / np.linalg.norm(np.linalg.inv(K_star).dot(h1))
        E = np.sign(np.linalg.inv(K_star).dot(h3)[0])
        alpha = alpha_abs * E
        r1 = alpha * np.linalg.inv(K_star).dot(h1)
        r2 = alpha * np.linalg.inv(K_star).dot(h2)
        r3 = np.cross(r1, r2)
        r_star = np.array([r1, r2, r3])
        t_star = alpha * np.linalg.inv(K_star).dot(h3)
        m = np.matmul(K_star, np.array([[r_star[0][0],r_star[0][1],r_star[0][2],t_star[0]], [r_star[1][0],r_star[1][1],r_star[1][2],t_star[1]], [r_star[2][0],r_star[2][1],r_star[2][2],t_star[2]]]))
        T_star.append(t_star)
        R_star.append(r_star)
        M.append(m)
        
    T_star = np.array(T_star)
    R_star = np.array(R_star)
    M = np.array(M)
    
    H_new = list()
    
    for h in H:
        h1 = h[:,0]
        h2 = h[:,1]
        h3 = h[:,2]
        
        h = np.array([h1.T, h2.T, np.array([0,0,0]), h3.T]).T
        H_new.append(h)
    
    H = np.array(H_new)
    
    return u0, v0, alpha_u, alpha_v, s, K_star, T_star, R_star, M, H

In [ ]:
def mean_square_error(M, world_points, img_points):
    m1T = M[0][:4]
    m2T = M[1][:4]
    m3T = M[2][:4]
    mse = 0.0
    for wp, ip in zip(world_points, img_points):
        xi = ip[0]
        yi = ip[1]
        pi = np.array(wp)
        pi = np.concatenate([pi, [1]]).T
        p_xi = (m1T.dot(pi)) / (m3T.dot(pi))
        p_yi = (m2T.dot(pi)) / (m3T.dot(pi))
        mse += (((xi - p_xi) ** 2) + ((yi - p_yi) ** 2))
    mse = mse / len(world_points)
    return mse

In [ ]:
world_points, img_points = get_correspondence_data(correspondence_points_files=correspondence_points_files)
print(f'number of points : {world_points.shape} , {img_points.shape}')
print()

ret, matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(world_points, img_points, cv2.cvtColor(load_image(image_file=image_files[0]), cv2.COLOR_BGR2GRAY).shape[::-1], None, None)
print(f'opencv calibratecamera output')
print(f'matrix :\n {matrix}')
print(f'distortion :\n {distortion}')
print(f'r_vecs :\n {r_vecs}')
print(f't_vecs :\n {t_vecs}')
print()

print(f'program output')
if is_planner == False:
    
    world_points = world_points[0]
    img_points = img_points[0]
    
    A = get_matirx_A_NP(world_points=world_points, img_points=img_points)
    print(f'shape of A : {A.shape}')
    print()
    
    a1, a2, a3, b, M = get_matrix_M_NP(A=A)
    print(f'a1 :\n {a1}')
    print(f'a2 :\n {a2}')
    print(f'a3 :\n {a3}')
    print(f'b :\n {b}')
    print(f'M :\n {M}')
    print()
    
    u0, v0, alpha_u, alpha_v, s, K_star, T_star, R_star, M = compute_parameters_NP(a1=a1, a2=a2, a3=a3, b=b)
    print(f'u0 : {u0}')
    print(f'v0 : {v0}')
    print(f'alpha_u : {alpha_u}')
    print(f'alpha_v : {alpha_v}')
    print(f's : {s}')
    print(f'K_star :\n {K_star}')
    print(f'T_star :\n {T_star}')
    print(f'R_star :\n {R_star}')
    print(f'M :\n {M}')
    print()
    
    mse = mean_square_error(M=M, world_points=world_points, img_points=img_points)
    print(f'mean square error = {mse}')
    print()

elif is_planner == True:
        
    A = get_matirx_A_P(world_points=world_points, img_points=img_points)
    print(f'shape of A : {A.shape}')
    print()
    
    H = get_matrix_H_P(A=A)
    print(f'H :\n {H}')
    print()
        
    V = get_matrix_V_P(H=H)
    print(f'V : \n {V}')
    print()
    
    u0, v0, alpha_u, alpha_v, s, K_star, T_star, R_star, M, H = compute_parameters_P(H=H, V=V)
    print(f'u0 : {u0}')
    print(f'v0 : {v0}')
    print(f'alpha_u : {alpha_u}')
    print(f'alpha_v : {alpha_v}')
    print(f's : {s}')
    print(f'K_star :\n {K_star}')
    print(f'T_star :\n {T_star}')
    print(f'R_star :\n {R_star}')
    print(f'M :\n {M}')
    print(f'H :\n {H}')
    print()
    
    mses = list()
    for i in range(H.shape[0]):
        mse = mean_square_error(M=H[i], world_points=world_points[i], img_points=img_points[i])
        mses.append(mse)
    print(f'mean square errors = {mses}')
    print()

# Part-3

In [ ]:
def get_ransac_config(config_file):
    with open(config_file, 'r') as file:
        P = float(file.readline().split()[0])
        W = float(file.readline().split()[0])
        N = int(file.readline().split()[0])
        D = int(file.readline().split()[0])
        K = int(file.readline().split()[0])
        K_upper = int(file.readline().split()[0])
        T = int(file.readline().split()[0])
    return P, W, N, D, K, K_upper, T

In [ ]:
def get_distance_NP(M, world_points, img_points):
    
    m1T = M[0][:4]
    m2T = M[1][:4]
    m3T = M[2][:4]
    
    distances = list()
    
    for wp,ip in zip(world_points, img_points):
        xi = ip[0]
        yi = wp[1]
        pi = np.array(wp)
        pi = np.concatenate([pi, [1]]).T
        p_xi = (m1T.dot(pi)) / (m3T.dot(pi))
        p_yi = (m2T.dot(pi)) / (m3T.dot(pi))
        distance = np.sqrt((((xi - p_xi) ** 2) + ((yi - p_yi) ** 2)))
        distances.append(distance)
    
    return distances

In [ ]:
def ransac(world_points, img_points, P, W, N, D, K, K_upper, T):
    
    best_model = None
    current_K = 0
    current_inliners = D
    A = get_matirx_A_NP(world_points=world_points, img_points=img_points)
    a1, a2, a3, b, M = get_matrix_M_NP(A=A)
    distances = get_distance_NP(M=M, world_points=world_points, img_points=img_points)
    median_distance = np.median(distances)
    T = 1.5 * median_distance
    
    while(current_K < K):
        
        selected_indices = np.random.choice(world_points.shape[0], N)
        random_world_points, random_img_points = world_points[selected_indices], img_points[selected_indices]
                
        A = get_matirx_A_NP(world_points=random_world_points, img_points=random_img_points)
        a1, a2, a3, b, M = get_matrix_M_NP(A=A)
        
        distances = get_distance_NP(M=M, world_points=world_points, img_points=img_points)
        
        inliners = list()
        
        for index, distance in zip(range(len(distances)), distances):
            if distance < T:
                inliners.append(index)
                
        if len(inliners) >= current_inliners:
            current_inliners = len(inliners)
            inliner_world_points, inliner_img_points = world_points[inliners], img_points[inliners]
            A = get_matirx_A_NP(world_points=inliner_world_points, img_points=inliner_img_points)
            a1, a2, a3, b, M = get_matrix_M_NP(A=A)
            distances = get_distance_NP(M=M, world_points=world_points, img_points=img_points)
            best_model = M
        
        median_distance = np.median(distances)
        T = 1.5 * median_distance
        W = len(inliners)/world_points.shape[0]
        K = np.absolute(math.log(1 - P) / math.log(1 - (W ** N)))
        K = K_upper if K >= K_upper else K
        current_K += 1;
    
    return best_model, K, W, T

In [ ]:
world_points, img_points = get_correspondence_data(correspondence_points_files=correspondence_points_files)
world_points = world_points[0]
img_points = img_points[0]
print(f'number of points : {world_points.shape} , {img_points.shape}')
print()

P, W, N, D, K, K_upper, T = get_ransac_config(config_file=ransac_config_file)
print(f'P : {P}')
print(f'W : {W}')
print(f'N : {N}')
print(f'D : {D}')
print(f'K : {K}')
print(f'K_upper : {K_upper}')
print(f'T : {T}')
print()

best_model, K, W, T = ransac(world_points=world_points, img_points=img_points, P=P, W=W, N=N, D=D, K=K, K_upper=K_upper, T=T)
print(f'Best Model :\n {best_model}')
print(f'K : {K}')
print(f'W : {W}')
print(f'T : {T}')
print()

a1 = best_model[0][:3].T
a2 = best_model[1][:3].T
a3 = best_model[2][:3].T
b = best_model[:,3].reshape((3,1))
u0, v0, alpha_u, alpha_v, s, K_star, T_star, R_star, M = compute_parameters_NP(a1=a1, a2=a2, a3=a3, b=b)
print(f'u0 : {u0}')
print(f'v0 : {v0}')
print(f'alpha_u : {alpha_u}')
print(f'alpha_v : {alpha_v}')
print(f's : {s}')
print(f'K_star :\n {K_star}')
print(f'T_star :\n {T_star}')
print(f'R_star :\n {R_star}')
print(f'M :\n {M}')
print()

mse = mean_square_error(M=M, world_points=world_points, img_points=img_points)
print(f'mean square error = {mse}')
print()